In [1]:
from pulp import *   # PuLP - Python Library for Linear Programming
import pandas as pd
import numpy as np
import math

In [2]:
n_supply = 100   #number of supply points
n_demand = 4096  #number of demand points

#Importing csv files 
demand_set = pd.read_csv('Prediction.csv') 
supply_set = pd.read_csv('exisiting_EV_infrastructure_2018.csv')
distij=pd.read_csv('distijtrans.csv')

In [3]:
#storing the distance matrix in 2D array
dist_matrix=distij.to_numpy()
df1=pd.DataFrame(dist_matrix)
df1

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,53.142156,52.208836,51.278030,50.349877,49.424527,48.502141,47.582890,46.666961,45.754552,44.845878,...,44.300780,44.409828,44.541067,44.694300,44.869301,45.065818,45.283571,45.522253,45.781539,46.061080
1,68.387858,67.854449,67.331667,66.819761,66.318983,65.829588,65.351830,64.885968,64.432258,63.990961,...,17.987223,18.943792,19.904632,20.869151,21.836863,22.807360,23.780302,24.755401,25.732411,26.711122
2,73.397670,72.772244,72.155256,71.546925,70.947474,70.357129,69.776122,69.204687,68.643064,68.091495,...,9.817786,10.628756,11.469612,12.334242,13.217983,14.117245,15.029243,15.951792,16.883164,17.821974
3,62.558091,62.084277,61.623049,61.174692,60.739490,60.317729,59.909692,59.515661,59.135917,58.770736,...,25.309363,26.258458,27.211199,28.167217,29.126188,30.087831,31.051898,32.018168,32.986449,33.956569
4,61.656645,60.831721,60.012122,59.198068,58.389791,57.587534,56.791553,56.002116,55.219502,54.444006,...,28.539067,28.660684,28.816510,29.005994,29.228481,29.483225,29.769397,30.086100,30.432382,30.807244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,49.423023,48.514871,47.610402,46.709827,45.813377,44.921299,44.033859,43.151342,42.274057,41.402337,...,43.449039,43.667798,43.908247,44.170032,44.452777,44.756083,45.079536,45.422705,45.785147,46.166409
96,44.393460,43.728052,43.075582,42.436646,41.811866,41.201885,40.607371,40.029012,39.467518,38.923620,...,38.619456,39.253351,39.902240,40.565404,41.242153,41.931830,42.633808,43.347489,44.072304,44.807714
97,41.812426,40.961461,40.117373,39.280605,38.451635,37.630979,36.819192,36.016874,35.224672,34.443284,...,45.132448,45.546682,45.978936,46.428707,46.895492,47.378788,47.878093,48.392914,48.922760,49.467149
98,56.712784,55.968438,55.232167,54.504297,53.785171,53.075143,52.374583,51.683877,51.003424,50.333641,...,27.887728,28.316983,28.774602,29.259253,29.769616,30.304393,30.862314,31.442146,32.042702,32.662837


In [4]:
#Demand values of 2018
ev_demands=np.array(demand_set['2019'])
ev_demands

array([15.08853972, 13.45382863, 17.50160494, ...,  3.78593664,
        7.28377172,  7.93874021])

In [19]:
dsij_matrix=pd.read_csv('dsij.csv',header=None)
dsij_matrix=dsij_matrix.to_numpy()
dsij_matrix

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 2.2185186, 5.2625738,
        5.8609394],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

#Smax value of each supply point
supply_max=np.array(supply_set['smax'])
supply_max

In [25]:
#Initialize the linear optimization model
model = LpProblem("Supply-Demand-Problem", LpMinimize)
model

Supply-Demand-Problem:
MINIMIZE
None
VARIABLES

In [7]:
#Creating unique index number for each value in DSij matrix
index_numbers = [str(i) for i in range(100)]
index_numbers.sort()
print("Variable Indices:", index_numbers)

Variable Indices: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


In [8]:
#Creating unique variables using index numbers
DV_variables = LpVariable.matrix("X", index_numbers, cat = "Integer", lowBound= 0 )
smax = np.array(DV_variables).reshape(n_supply)
print("Decision Variable/Allocation Matrix: ")
print(smax)

Decision Variable/Allocation Matrix: 
[X_0 X_1 X_10 X_11 X_12 X_13 X_14 X_15 X_16 X_17 X_18 X_19 X_2 X_20 X_21
 X_22 X_23 X_24 X_25 X_26 X_27 X_28 X_29 X_3 X_30 X_31 X_32 X_33 X_34 X_35
 X_36 X_37 X_38 X_39 X_4 X_40 X_41 X_42 X_43 X_44 X_45 X_46 X_47 X_48 X_49
 X_5 X_50 X_51 X_52 X_53 X_54 X_55 X_56 X_57 X_58 X_59 X_6 X_60 X_61 X_62
 X_63 X_64 X_65 X_66 X_67 X_68 X_69 X_7 X_70 X_71 X_72 X_73 X_74 X_75 X_76
 X_77 X_78 X_79 X_8 X_80 X_81 X_82 X_83 X_84 X_85 X_86 X_87 X_88 X_89 X_9
 X_90 X_91 X_92 X_93 X_94 X_95 X_96 X_97 X_98 X_99]


df=pd.DataFrame(dsij_mat)
df

In [20]:
#Defining the Customer Dissatisfaction cost
cost_eq = lpSum(dsij_matrix*dist_matrix)

In [28]:
#Adding the cost equation to the model
model +=  cost_eq
print(model)

Supply-Demand-Problem:
MINIMIZE
2017246.7404462774
SUBJECT TO
Supply_Constraints_0: - X_0 <= -2199.9999998

Supply_Constraints_1: - X_1 <= -3600.0000113

Supply_Constraints_2: - X_10 <= -6211.63792721

Supply_Constraints_3: - X_11 <= -2999.999992

Supply_Constraints_4: - X_12 <= -4600.00002243

Supply_Constraints_5: - X_13 <= -1156.76732055

Supply_Constraints_6: - X_14 <= -4999.999997

Supply_Constraints_7: - X_15 <= -3599.999998

Supply_Constraints_8: - X_16 <= -2200

Supply_Constraints_9: - X_17 <= -3200.000003

Supply_Constraints_10: - X_18 <= -1200.0000026

Supply_Constraints_11: - X_19 <= -3399.99999

Supply_Constraints_12: - X_2 <= -2999.999991

Supply_Constraints_13: - X_20 <= -3400.000008

Supply_Constraints_14: - X_21 <= -5599.99999947

Supply_Constraints_15: - X_22 <= -3399.99999725

Supply_Constraints_16: - X_23 <= -1599.99999584

Supply_Constraints_17: - X_24 <= -799.999994

Supply_Constraints_18: - X_25 <= -3399.999982

Supply_Constraints_19: - X_26 <= -4399.9999995

Supp

In [27]:
#Supply Constraint
#Column summation must be less than or equal to Smax 
for i in range(n_supply):
    print(lpSum(dsij_matrix[i][j] for j in range(n_demand)) <= smax[i])
    model += lpSum(dsij_matrix[i][j] for j in range(n_demand)) <= smax[i] , "Supply Constraints " + str(i)

-X_0 <= -2199.9999997999994
-X_1 <= -3600.000011299999
-X_10 <= -6211.637927209996
-X_11 <= -2999.9999920000005
-X_12 <= -4600.00002243
-X_13 <= -1156.7673205500005
-X_14 <= -4999.999996999999
-X_15 <= -3599.999998
-X_16 <= -2199.9999999999995
-X_17 <= -3200.0000029999997
-X_18 <= -1200.0000026000005
-X_19 <= -3399.9999899999993
-X_2 <= -2999.9999910000006
-X_20 <= -3400.000008
-X_21 <= -5599.999999470001
-X_22 <= -3399.999997250001
-X_23 <= -1599.99999584
-X_24 <= -799.9999940000001
-X_25 <= -3399.999982
-X_26 <= -4399.999999499999
-X_27 <= -4399.999999999999
-X_28 <= -5200.0000015000005
-X_29 <= -2600.000013
-X_3 <= -3600.0000012130017
-X_30 <= -2600.0000025999984
-X_31 <= -5200.000004
-X_32 <= -5799.999989000001
-X_33 <= -3199.999997985
-X_34 <= -4599.999999
-X_35 <= -2400.0000118499997
-X_36 <= -1800.0000149999998
-X_37 <= -4799.99999684
-X_38 <= -5399.999993
-X_39 <= -4000.000015899999
-X_4 <= -1400.0000006
-X_40 <= -799.9999999999999
-X_41 <= -4400.000002700001
-X_42 <= -2399.999

In [29]:
# Demand Constraints
#Row summation must be equal to demand forecast
for j in range(n_demand):
    print(lpSum(dsij_matrix[i][j] for i in range(n_supply)) >= ev_demands[j])
    model += lpSum(dsij_matrix[i][j] for i in range(n_supply)) >= ev_demands[j] , "Demand Constraints " + str(j)

0 >= 2.96896772
0 >= 2.4337376299999995
0 >= 4.483350939999999
0 >= 4.66229622
0 >= 5.17060296
0 >= 6.301892970000001
0 >= 5.040822900000002
0 >= 5.417298299999999
0 >= 5.380621699999999
0 >= 7.39389577
0 >= 7.308127129999999
0 >= 8.047849300000003
0 >= 4.2158571500000015
0 >= 6.624962619999998
0 >= 5.813018019999998
0 >= 7.156415200000001
0 >= 6.687080849999997
0 >= 6.835466140000001
0 >= 6.078270310000001
0 >= 4.881993029999997
0 >= 6.5838088
0 >= 4.72980059
0 >= 6.9510597700000005
0 >= 5.967190710000001
0 >= 5.270221409999998
0 >= 3.727242180000001
0 >= 6.032899999999998
0 >= 5.5681798199999974
0 >= 4.215387190000001
0 >= 3.4711194600000006
0 >= 6.4237987300000015
0 >= 9.08473168
0 >= 8.754865089999996
0 >= 9.755984699999999
0 >= 9.20048808
0 >= 9.61795926
0 >= 10.39146289
0 >= 10.665908850000001
0 >= 8.91814523
0 >= 12.342291750000001
0 >= 15.631408000000008
0 >= 11.55061126000001
0 >= 6.49899971
0 >= 10.541964789999994
0 >= 9.129195760000002
0 >= 6.775679029999996
0 >= 6.608822150

0 >= 20.069569599999994
0 >= 10.49148023
0 >= 10.887076620000002
0 >= 18.829511499999995
0 >= 16.104014600000014
0 >= 23.666906299999994
0 >= 19.08404859999999
0 >= 8.90642489999999
0 >= 21.793315000000007
0 >= 14.177851160000003
0 >= 23.55107860000001
0 >= 3.175329699999992
0 >= 27.1228969
0 >= 12.508032700000001
0 >= 4.470583199999993
0 >= 17.10028641000001
0 >= 12.630674139999996
0 >= 1.4761297829999998
0 >= 8.945614290000002
0 >= 15.662084530000001
0 >= 11.891643299999998
0 >= 1.187701761
0 >= 1.213540262
0 >= 1.18306366
0 >= 1.2508815229999999
0 >= 1.382485393
0 >= 1.035390601
0 >= 1.163905755
0 >= 1.408572313
0 >= 0.7586142849999997
0 >= 15.776819429999996
0 >= 23.5085694
0 >= 19.82939019999999
0 >= 27.391442799999993
0 >= 29.35884200000001
0 >= 21.55245079999999
0 >= 12.281633749999997
0 >= 6.1009083500000045
0 >= 13.574489069999998
0 >= 14.041449530000001
0 >= 12.021173529999999
0 >= 9.825746970000004
0 >= 6.003374350000001
0 >= 12.951606289999994
0 >= 21.72790119999999
0 >= 19

0 >= 8.09442747
0 >= 10.22732757
0 >= 18.623895599999997
0 >= 11.665422500000005
0 >= 30.841761100000014
0 >= 54.262236099999996
0 >= 58.060380400000014
0 >= 25.008421800000008
0 >= 45.993170700000036
0 >= 23.30682490000001
0 >= 26.79933889999998
0 >= 23.592300100000017
0 >= 35.9624517
0 >= 53.833815699999974
0 >= 57.5422815
0 >= 35.46016160000002
0 >= 31.88693409999999
0 >= 18.616954800000002
0 >= 39.67047739999998
0 >= 44.42018830000001
0 >= 2.8379130000000146
0 >= 2.4522514870000007
0 >= 2.1178078320000004
0 >= 2.1141221350000006
0 >= 1.9508935270000003
0 >= 2.6193291789999993
0 >= 6.7950604299999995
0 >= 6.782205620000003
0 >= 7.8871568100000005
0 >= 8.444155870000003
0 >= 7.211735520000001
0 >= 9.863924449999999
0 >= 10.976721100000006
0 >= 8.14529589
0 >= 11.727149959999998
0 >= 10.42084886
0 >= 4.46329351
0 >= 2.68167914
0 >= 6.959339299999996
0 >= 9.2425845
0 >= -0.0
0 >= 15.255718599999994
0 >= 15.423742899999993
0 >= 25.469497900000007
0 >= 19.93929300000002
0 >= 13.221585199

0 >= 1.429775715
0 >= 1.383304913
0 >= 1.056628355
0 >= 1.7273623650000003
0 >= 10.865965309999993
0 >= 9.665410250000008
0 >= 13.399469409999995
0 >= 4.5415441800000025
0 >= 10.911492410000001
0 >= 7.9648615199999995
0 >= 12.881544380000008
0 >= 9.89182538
0 >= 8.626175950000004
0 >= 20.550501550000007
0 >= 19.133851300000003
0 >= 15.904994799999997
0 >= 15.437333930000008
0 >= 11.429784259999998
0 >= 18.956642200000005
0 >= 21.295612899999995
0 >= 26.163116700000003
0 >= 20.547564299999976
0 >= 33.058104799999995
0 >= 30.265276099999994
0 >= 12.064669500000008
0 >= 7.748562930000006
0 >= 23.285851100000002
0 >= 20.58400689999999
0 >= 38.8070099
0 >= -10.611659999999972
0 >= 48.49342759999999
0 >= 46.44104140000002
0 >= 20.65145480000001
0 >= 5.355607099999986
0 >= -0.03807399999999461
0 >= 11.645211999999987
0 >= -5.330321599999991
0 >= 21.151733400000012
0 >= 15.236310300000014
0 >= -10.064852299999984
0 >= 8.147906199999994
0 >= 40.35504549999996
0 >= 34.29730270000002
0 >= 17.8585

0 >= 1.6766710799999998
0 >= 1.0790002909999998
0 >= 1.1259712899999998
0 >= 6.1944424
0 >= 7.51081967
0 >= -0.0
0 >= 7.535590080000002
0 >= 6.237697370000003
0 >= 14.821035369999997
0 >= 16.91871124
0 >= 13.711376309999999
0 >= 14.09593571
0 >= 16.417297750000003
0 >= 13.74988261
0 >= 13.534329540000002
0 >= 19.08974296999999
0 >= 32.870326199999994
0 >= 16.877808799999997
0 >= -0.2519750999999957
0 >= 5.560944499999998
0 >= 19.563457220000004
0 >= 11.667769849999999
0 >= 23.817046000000005
0 >= 28.437920599999984
0 >= 22.308706
0 >= 40.65351909999998
0 >= 33.77967510000002
0 >= -0.3456378999999856
0 >= 10.609904999999998
0 >= 16.398039100000005
0 >= -2.3063654000000042
0 >= 5.575023600000009
0 >= 13.096410399999996
0 >= 39.81666959999998
0 >= 51.34296470000004
0 >= 43.0748806
0 >= 17.756209800000022
0 >= 21.857931400000012
0 >= 40.154790399999996
0 >= 50.983992099999995
0 >= 63.72803139999999
0 >= -4.561568399999999
0 >= 55.9743919
0 >= 41.16556560000001
0 >= 22.59685909999999
0 >= 3

0 >= 2.163155302
0 >= 1.9553145420000009
0 >= 2.9001829889999993
0 >= 2.2149501869999995
0 >= 2.250772369
0 >= 1.7159170780000004
0 >= 1.4683724649999998
0 >= 1.489462059
0 >= 1.261376374
0 >= 1.5531630020000002
0 >= 2.1882959510000006
0 >= 2.8709852599999994
0 >= 0.8236222470000003
0 >= 3.0244723900000015
0 >= 2.5379042100000007
0 >= 2.0480249300000004
0 >= 2.6569974
0 >= 2.362066349999999
0 >= 0.4824768800000001
0 >= 0.9891802360000002
0 >= 2.74790318
0 >= 3.0295435400000006
0 >= 2.17271575
0 >= 2.62442373
0 >= 3.52412288
0 >= 1.427145898
0 >= 1.3382586759999997
0 >= 0.6171298829999996
0 >= 1.3039001260000012
0 >= 2.71473791
0 >= 3.0095183500000005
0 >= 1.6626114699999999
0 >= 2.229940469999999
0 >= 3.1459461499999994
0 >= 1.6653992329999996
0 >= 1.9775758640000003
0 >= 1.9734466680000002
0 >= 1.6332623669999997
0 >= 1.4674543559999997
0 >= 2.1317700759999996
0 >= 2.1142428950000003
0 >= 1.4021551639999998
0 >= 1.567418042
0 >= 2.021197924
0 >= 2.0778008089999993


In [30]:
model

Supply-Demand-Problem:
MINIMIZE
2017246.7404462774
SUBJECT TO
Supply_Constraints_0: - X_0 <= -2199.9999998

Supply_Constraints_1: - X_1 <= -3600.0000113

Supply_Constraints_2: - X_10 <= -6211.63792721

Supply_Constraints_3: - X_11 <= -2999.999992

Supply_Constraints_4: - X_12 <= -4600.00002243

Supply_Constraints_5: - X_13 <= -1156.76732055

Supply_Constraints_6: - X_14 <= -4999.999997

Supply_Constraints_7: - X_15 <= -3599.999998

Supply_Constraints_8: - X_16 <= -2200

Supply_Constraints_9: - X_17 <= -3200.000003

Supply_Constraints_10: - X_18 <= -1200.0000026

Supply_Constraints_11: - X_19 <= -3399.99999

Supply_Constraints_12: - X_2 <= -2999.999991

Supply_Constraints_13: - X_20 <= -3400.000008

Supply_Constraints_14: - X_21 <= -5599.99999947

Supply_Constraints_15: - X_22 <= -3399.99999725

Supply_Constraints_16: - X_23 <= -1599.99999584

Supply_Constraints_17: - X_24 <= -799.999994

Supply_Constraints_18: - X_25 <= -3399.999982

Supply_Constraints_19: - X_26 <= -4399.9999995

Supp

In [ ]:
#Solve the model by minimizing the cost equation 
#by following the supply and demand constraints

model.solve(PULP_CBC_CMD())
status =  LpStatus[model.status]
print(status)

In [ ]:
#Total cost of customer dissatisfaction
print("Total Cost:", model.objective.value())

# DSij Variable's value
for v in model.variables():
    print(v.name,"=", v.value())

In [ ]:
#replacing the variable names with the actual value in DSij matrix

dsij_pred=np.zeros((100,4096))
p=0
lst=[]
for v in model.variables():
    k=v.value()
    lst.append(k)
    
for i in range(100):
    for j in range(4096):
        dsij_pred[i][j]=lst[p]
        p+=1
        
table=pd.DataFrame(dsij_pred)
table

In [ ]:
#Checking the Demand constraint (row summation)
sum(table[0])

In [ ]:
#Comparing with demand value
ev_demands[0]

In [ ]:
#Checking the Supply constraint (column summation)
sum(table.iloc[0,:].values)

In [ ]:
#Comparing with supply value
supply_max[0]